In [10]:
# Load packages and settings
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.min_rows', 50)
import seaborn as sns


import matplotlib.pyplot as plt
%matplotlib widget
plt.rcParams['figure.figsize'] = (12,8)
plt.rcParams["image.cmap"] = "tab10"
plt.rcParams['axes.prop_cycle'] = plt.cycler(color=plt.cm.tab10.colors)
fs_label = 15
parameters = {
                'axes.labelsize': fs_label,
                'axes.titlesize': fs_label+4,
                'xtick.labelsize': fs_label,
                'ytick.labelsize': fs_label, 
                'legend.fontsize': fs_label, 
                'lines.markersize': 10,
                'lines.linewidth': 3
             }
plt.rcParams.update(parameters)
%matplotlib widget
from matplotlib import cm # Colormaps
import matplotlib.colors as colors
# cmap = plt.cm.get_cmap('Dark2',len(ageGroups))

import locale
import matplotlib.dates as mdates
locale.setlocale(locale.LC_TIME,"Danish")
# ax1.xaxis.set_major_formatter(mdates.DateFormatter('%b\n%Y'))

import os
# import csv
import math

from datetime import date


saveFigures = True
# saveFigures = False
print('saveFigures is set to: '+str(saveFigures))

print('Done loading packages')

# Define running mean functions
def rnMean(data,meanWidth):
    return np.convolve(data, np.ones(meanWidth)/meanWidth, mode='valid')
def rnTime(t,meanWidth):
    return t[math.floor(meanWidth/2):-math.ceil(meanWidth/2)+1]
    
    
# Define paths
rootdir_data = os.getcwd() +"\\..\\DanskeData\\" 

path_data = rootdir_data + "ssi_data\\"
path_dash = rootdir_data + "ssi_dashboard\\"
path_vacc = rootdir_data + "ssi_vacc\\"
path_owid = os.getcwd() +"\\..\\InternationalData\\data\\" 

path_figs = os.getcwd() +"\\..\\Figures\\" 

saveFigures is set to: True
Done loading packages


In [11]:
df = pd.read_csv(path_owid+'OWID_data.csv')

In [18]:
# df.columns
df.location.unique()

curCountries = ['Denmark','European Union','Estonia','Latvia','Lithuania','Germany','France','Italy','Spain','Portugal','United Kingdom','United States']
# curCountries = ['Denmark','Estonia','Latvia','Lithuania','Germany','France','Italy','Spain','Portugal','United Kingdom','United States']

curCountries = np.array(curCountries)

In [25]:
curCountry = curCountries[0]

vaccRates = []
infRates = []
for curCountry in curCountries:
    curdf = df[df.location == curCountry]

    # curLastRow = curdf[curdf.date == (curdf.date.max())]
    # curdf.people_vaccinated_per_hundred
    curVacc = curdf.people_vaccinated_per_hundred
    latestVaccRate = curVacc[~curVacc.isna()].iloc[-1]
    vaccRates.append(latestVaccRate)
    
    curInf = curdf.total_cases_per_million
    latestInfRate = curInf[~curInf.isna()].iloc[-1]
    infRates.append(100*latestInfRate/1000000) # Per million, changed to per hundred
    

vaccRates = np.array(vaccRates)    
infRates = np.array(infRates)

newOrder = np.argsort(vaccRates)

vaccRates = vaccRates[newOrder]
infRates = infRates[newOrder] 
curCountries = curCountries[newOrder]

immuRateDependent = vaccRates + infRates
immuRateIndependent = vaccRates + infRates - (vaccRates/100)*(infRates/100)*100
infRateIndependent = infRates - (vaccRates/100)*(infRates/100)*100

In [27]:


fig,ax1 = plt.subplots()

fig.patch.set_facecolor('lightgrey')
ax1.set_facecolor('lightgrey')

cmap = plt.cm.get_cmap('magma',5)
ax1.bar(curCountries,vaccRates,color=cmap(0),label='Vaccinations')
ax1.bar(curCountries,infRates,bottom=vaccRates,color=cmap(2),label='Cases (Dependent)')
ax1.bar(curCountries,infRateIndependent,bottom=vaccRates,color=cmap(1),label='Cases (Independent)')
# ax1.bar(curCountries,infRates,bottom=vaccRates,color=cmap(1),label='Cases (Dependent)')
# ax1.bar(curCountries,infRates - (vaccRates/100)*(infRates/100)*100,bottom=vaccRates,color=cmap(2),label='Cases (Independent)')
# ax1.bar(curCountries,vaccRates,color=cmap(0),label='Vaccinationer')
# ax1.bar(curCountries,infRates,bottom=vaccRates,color=cmap(1),label='Smittetilfælde (afhængig)')
# ax1.bar(curCountries,infRates - (vaccRates/100)*(infRates/100)*100,bottom=vaccRates,color=cmap(2),label='Smittetilfælde (uafhængig)')
# ax1.bar(curCountries,vaccRates,color='b',label='Vaccinationer')
# ax1.bar(curCountries,infRates,bottom=vaccRates,color='r',label='Smittetilfælde (afhængig)')
# ax1.bar(curCountries,infRates - (vaccRates/100)*(infRates/100)*100,bottom=vaccRates,color='m',label='Smittetilfælde (uafhængig)')

R0_wuhan = 2.7
R0_alpha = R0_wuhan*1.5
R0_delta = 4.7
# R0_delta_winter = 4.7*1.2
R0_delta_winter = R0_wuhan * 1.5 * 1.5

herd_wuhan = 1 - (1/R0_wuhan)
herd_alpha = 1 - (1/R0_alpha)
herd_delta = 1 - (1/R0_delta)
herd_delta_winter = 1 - (1/R0_delta_winter)



cmap = plt.cm.get_cmap('cividis',4)
# ax1.plot([-1,len(curCountries)],[100*herd_wuhan,100*herd_wuhan],color=cmap(0),label=f'Flokimmunitet, R0={R0_wuhan:1.1f}\nOriginal virus')
# ax1.plot([-1,len(curCountries)],[100*herd_alpha,100*herd_alpha],color=cmap(1),label=f'Flokimmunitet, R0={R0_alpha:1.1f}\nAlpha-variant')
# ax1.plot([-1,len(curCountries)],[100*herd_delta,100*herd_delta],color=cmap(2),label=f'Flokimmunitet, R0={R0_delta:1.1f}\nDelta-variant (sommer)')
# ax1.plot([-1,len(curCountries)],[100*herd_delta_winter,100*herd_delta_winter],color=cmap(3),label=f'Flokimmunitet, R0={R0_delta_winter:1.1f}\nDelta-variant (vinter)')
ax1.plot([-1,len(curCountries)],[100*herd_wuhan,100*herd_wuhan],color=cmap(0),label=f'Herd immunity, R0={R0_wuhan:1.1f}\nOriginal virus')
ax1.plot([-1,len(curCountries)],[100*herd_alpha,100*herd_alpha],color=cmap(1),label=f'Herd immunity, R0={R0_alpha:1.1f}\nAlpha-variant')
ax1.plot([-1,len(curCountries)],[100*herd_delta,100*herd_delta],color=cmap(2),label=f'Herd immunity, R0={R0_delta:1.1f}\nDelta-variant (summer)')
ax1.plot([-1,len(curCountries)],[100*herd_delta_winter,100*herd_delta_winter],color=cmap(3),label=f'Herd immunity, R0={R0_delta_winter:1.1f}\nDelta-variant (winter)')

ax1.legend(bbox_to_anchor=(1.04,0.5), loc="center left")
ax1.set_ylim([0,100])
ax1.set_ylim([50,100])
ax1.set_xlim(-0.5,len(curCountries)-0.5)
ax1.set_ylabel('Total rate of immunity in society [%]')
# ax1.set_ylabel('Grad af immunitet i samfundet')

from matplotlib.ticker import (MultipleLocator, AutoMinorLocator)
ax1.yaxis.set_minor_locator(MultipleLocator(5)) 
ax1.set_axisbelow(True)
ax1.grid(axis='y',which='both',zorder=0)

plt.xticks(rotation=90)
plt.tight_layout()

plt.savefig(path_figs + 'ImmunityComparison')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [83]:
fig,ax1 = plt.subplots()

fig.patch.set_facecolor('lightgrey')
ax1.set_facecolor('lightgrey')


cmap = plt.cm.get_cmap('magma',5)
ax1.barh(curCountries[::-1],vaccRates[::-1],height=0.5,edgecolor='k',color=cmap(0),label='Vaccinations')
ax1.barh(curCountries[::-1],infRates[::-1],left=vaccRates[::-1],height=0.5,edgecolor='k',color=cmap(2),label='Cases (Dependent)')
ax1.barh(curCountries[::-1],infRateIndependent[::-1],left=vaccRates[::-1],height=0.5,edgecolor='k',color=cmap(1),label='Cases (Independent)')

R0_wuhan = 2.7
R0_alpha = R0_wuhan*1.5
R0_delta = 4.7
# R0_delta_winter = 4.7*1.2
R0_delta_winter = R0_wuhan * 1.5 * 1.5
R0_delta_winter2 = R0_wuhan * 1.5 * 1.6

herd_wuhan = 1 - (1/R0_wuhan)
herd_alpha = 1 - (1/R0_alpha)
herd_delta = 1 - (1/R0_delta)
herd_delta_winter = 1 - (1/R0_delta_winter)
herd_delta_winter2 = 1 - (1/R0_delta_winter2)

# ax1.fill([0,20],[0,80],'k')
numCoun = len(curCountries)
cmap = plt.cm.get_cmap('cividis',5)
ax1.fill_between([0,100*herd_wuhan],[numCoun+1,numCoun+1],[-1,-1],color=cmap(1),zorder=-1,hatch='///',edgecolor='k',label=f'Flokimmunitet, R0={R0_wuhan:1.1f}\nOriginal virus') 
ax1.fill_between([0,100*herd_alpha],[numCoun+1,numCoun+1],[-1,-1],color=cmap(2),zorder=-2,hatch='\\',edgecolor='k',label=f'Flokimmunitet, R0={R0_alpha:1.1f}\nAlpha-variant')
ax1.fill_between([0,100*herd_delta],[numCoun+1,numCoun+1],[-1,-1],color=cmap(3),zorder=-3,hatch='x',edgecolor='k',label=f'Flokimmunitet, R0={R0_delta:1.1f}\nDelta-variant (sommer)')
ax1.fill_between([0,100*herd_delta_winter],[numCoun+1,numCoun+1],[-1,-1],color=[1,0.2,0.2],zorder=-4,hatch='/',edgecolor='k',label=f'Flokimmunitet, R0={R0_delta_winter:1.1f}\nDelta-variant (vinter)')
# ax1.fill_between([0,100*herd_delta_winter2],[numCoun+1,numCoun+1],[-1,-1],color=[1,0.2,0.2,0.5],zorder=-5,hatch='/')
# ax1.fill_between([0,100*herd_delta_winter2 + 5],[numCoun+1,numCoun+1],[-1,-1],color=[1,0.2,0.2,0.25],zorder=-6,hatch='/')

ax1.set_ylim([-1,numCoun])

# cmap = plt.cm.get_cmap('cividis',4)
# # ax1.plot([-1,len(curCountries)],[100*herd_wuhan,100*herd_wuhan],color=cmap(0),label=f'Flokimmunitet, R0={R0_wuhan:1.1f}\nOriginal virus') 
# # ax1.plot([-1,len(curCountries)],[100*herd_alpha,100*herd_alpha],color=cmap(1),label=f'Flokimmunitet, R0={R0_alpha:1.1f}\nAlpha-variant')
# # ax1.plot([-1,len(curCountries)],[100*herd_delta,100*herd_delta],color=cmap(2),label=f'Flokimmunitet, R0={R0_delta:1.1f}\nDelta-variant (sommer)')
# # ax1.plot([-1,len(curCountries)],[100*herd_delta_winter,100*herd_delta_winter],color=cmap(3),label=f'Flokimmunitet, R0={R0_delta_winter:1.1f}\nDelta-variant (vinter)')
# ax1.plot([-1,len(curCountries)],[100*herd_wuhan,100*herd_wuhan],color=cmap(0),label=f'Herd immunity, R0={R0_wuhan:1.1f}\nOriginal virus')
# ax1.plot([-1,len(curCountries)],[100*herd_alpha,100*herd_alpha],color=cmap(1),label=f'Herd immunity, R0={R0_alpha:1.1f}\nAlpha-variant')
# ax1.plot([-1,len(curCountries)],[100*herd_delta,100*herd_delta],color=cmap(2),label=f'Herd immunity, R0={R0_delta:1.1f}\nDelta-variant (summer)')
# ax1.plot([-1,len(curCountries)],[100*herd_delta_winter,100*herd_delta_winter],color=cmap(3),label=f'Herd immunity, R0={R0_delta_winter:1.1f}\nDelta-variant (winter)')

ax1.legend(bbox_to_anchor=(1.04,0.5), loc="center left")
# ax1.set_ylim([0,100])
ax1.set_xlim([50,100])
# ax1.set_xlim(-0.5,len(curCountries)-0.5)
ax1.set_xlabel('Total rate of immunity in society [%]')
# # ax1.set_ylabel('Grad af immunitet i samfundet')

# from matplotlib.ticker import (MultipleLocator, AutoMinorLocator)
# ax1.yaxis.set_minor_locator(MultipleLocator(5)) 
# ax1.set_axisbelow(True)
# ax1.grid(axis='y',which='both',zorder=0)

# plt.xticks(rotation=90)
plt.tight_layout()

plt.savefig(path_figs + 'ImmunityComparison2')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …